In [2]:
import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import lightgbm as lgb
import warnings
from datetime import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
warnings.filterwarnings('ignore')


In [3]:
item_cat = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
items=pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
test=pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

In [4]:
items = items.merge(item_cat, on='item_category_id', how='left')
train=train.merge(shops, on="shop_id", how="left")
######  merged all files into train like this
train = train.merge(items, on='item_id', how='left')

In [5]:
#for_merchant = train.groupby("shop_id").agg({"item_cnt_day":"sum"}).reset_index()
#train = train.merge(for_merchant, on="shop_id",how="left")

In [6]:
#train["total_paid"] = train["item_price"]*train["item_cnt_day"]
#total_paids = train.groupby("shop_id").agg({"total_paid":"sum"}).reset_index()
#train = train.merge(total_paids, on="shop_id",how="left")

In [7]:
train["date"] = pd.to_datetime(train["date"])

train.shape

(2935849, 10)

In [8]:
train["date"].min(), train["date"].max()

(Timestamp('2013-01-01 00:00:00'), Timestamp('2015-12-10 00:00:00'))

In [9]:
col_list = ["date","shop_id","item_cnt_day","item_id"]
train = train[col_list]

In [10]:
df = pd.concat([train, test], sort=False)

In [11]:
#drop_list2=["total_sales_per_shop","total_paid_x","total_paid_y"]
#train.drop(drop_list2,axis=1,inplace=True)

In [12]:
#train[train["shop_id"]==5].sort_values("shop_id", ascending=True).tail(30)

In [13]:
#train = train.rename(columns=lambda x: x.replace('total_paid_x', 'total_sales'))
#train = train.rename(columns=lambda x: x.replace('total_paid_y', 'total_sales_per_shop'))

In [14]:
##delete shop_name, item_name, item_category_name
#drop_list = ["shop_name", "item_name", "item_category_name","item_price","date_block_num","item_category_id"]
#train.drop(drop_list, axis=1, inplace=True)
#test.drop(drop_list, axis=1, inplace=True)

In [15]:
#for id in train.shop_id.unique():
    #plt.figure(figsize=(15, 15))
   # plt.subplot(3, 1, 1, title = str(id) + ' 2013-2014 total_sales_per_shop')
    #train[(train.shop_id == id) &( train.date >= "2013-01-01" )& (train.date < "2014-01-01")]["item_cnt_day"].plot()
    #plt.xlabel('')
    #plt.subplot(3, 1, 2,title = str(id) + ' 2015-2016 Transaction')
    #train[(train.shop_id == id) &( train.date >= "2014-01-01" )& (train.date < "2015-11-10")]["item_cnt_day"].plot()
    #plt.xlabel('')
    #plt.show()

def create_date_features(df, date_column):
    df['month'] = df[date_column].dt.month
    df['day_of_month'] = df[date_column].dt.day
    df['day_of_year'] = df[date_column].dt.dayofyear
    df['week_of_year'] = df[date_column].dt.weekofyear
    df['day_of_week'] = df[date_column].dt.dayofweek
    df['year'] = df[date_column].dt.year
    df["is_wknd"] = df[date_column].dt.weekday // 4
    df['is_month_start'] =df[date_column].dt.is_month_start.astype(int)
    df['is_month_end'] = df[date_column].dt.is_month_end.astype(int)
    df['quarter'] = df[date_column].dt.quarter
    df['is_quarter_start'] = df[date_column].dt.is_quarter_start.astype(int)
    df['is_quarter_end'] = df[date_column].dt.is_quarter_end.astype(int)
    df['is_year_start'] = df[date_column].dt.is_year_start.astype(int)
    df['is_year_end'] = df[date_column].dt.is_year_end.astype(int)
    return df

In [16]:
df = create_date_features(df, "date")

In [17]:
df.head()

,date,shop_id,item_cnt_day,item_id,ID,month,day_of_month,day_of_year,week_of_year,day_of_week,year,is_wknd,is_month_start,is_month_end,quarter,is_quarter_start,is_quarter_end,is_year_start,is_year_end
0,2013-02-01,59,1.000,22154,NaN,2.000,1.000,32.000,5.000,4.000,2013.000,1.000,1,0,1.000,0,0,0,0
1,2013-03-01,25,1.000,2552,NaN,3.000,1.000,60.000,9.000,4.000,2013.000,1.000,1,0,1.000,0,0,0,0
2,2013-05-01,25,-1.000,2552,NaN,5.000,1.000,121.000,18.000,2.000,2013.000,0.000,1,0,2.000,0,0,0,0
3,2013-06-01,25,1.000,2554,NaN,6.000,1.000,152.000,22.000,5.000,2013.000,1.000,1,0,2.000,0,0,0,0
4,2013-01-15,25,1.000,2555,NaN,1.000,15.000,15.000,3.000,1.000,2013.000,0.000,0,0,1.000,0,0,0,0


In [18]:
# Lag/Shifted Features


def random_noise(dataframe):
    return np.random.normal(scale=1.6, size=(len(dataframe),))

def lag_features(dataframe, lags):
    for lag in lags:
        dataframe['sales_lag_' + str(lag)] = dataframe.groupby(["shop_id"])['item_cnt_day'].transform(
            lambda x: x.shift(lag)) + random_noise(dataframe)
    return dataframe


df = lag_features(df, [30,60,91,92,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,
                       350,351,352,352,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,
                       538,539,540,541,542,
                       718,719,720,721,722])




In [19]:
# Rolling Mean Features


def roll_mean_features(dataframe, windows):
    for window in windows:
        dataframe['sales_roll_mean_' + str(window)] = dataframe.groupby("shop_id")['item_cnt_day']. \
                                                          transform(
            lambda x: x.shift(1).rolling(window=window, min_periods=10, win_type="triang").mean()) + random_noise(
            dataframe)
    return dataframe

df = roll_mean_features(df, [365, 546])


In [20]:
# Exponentially Weighted Mean Features


def ewm_features(dataframe, alphas, lags):
    for alpha in alphas:
        for lag in lags:
            dataframe['sales_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = \
                dataframe.groupby("shop_id")['item_cnt_day'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
    return dataframe

alphas = [0.95, 0.9, 0.8, 0.7, 0.5]
lags = [30,60,91,92,178,179,180,181,182,359,360,361,449,450,451,539,540,541,629,630,631,720]

df = ewm_features(df, alphas, lags)

In [21]:
df["is_black_friday"] = 0
df.loc[df["date"].isin(["2013-11-22","2013-11-23","2014-11-29","2014-11-30"]) ,"is_black_friday"]=1

df["is_summer_solstice"] = 0
df.loc[df["date"].isin(["2013-06-19","2013-06-20","2013-06-21","2013-06-22",
                                    "2014-06-19","2014-06-20","2014-06-21","2014-06-22",]) ,"is_summer_solstice"]=1

In [22]:
#train = pd.get_dummies(train, columns=['shop_id','day_of_week', 'month'])
#train['total_sales'] = np.log1p(train["total_sales"].values)


In [23]:
def smape(preds, target):
    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds - target)
    denom = np.abs(preds) + np.abs(target)
    smape_val = (200 * np.sum(num / denom)) / n
    return smape_val

def lgbm_smape(preds, train_data):
    labels = train_data.get_label()
    smape_val = smape(np.expm1(preds), np.expm1(labels))
    return 'SMAPE', smape_val, False

In [24]:
import re
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

# 2015'nin 10.ayına kadar train seti.
train_ = df.loc[(df["date"] < "2015-10-01"), :]

# 2020'nin son 3 ayı validasyon seti.                ######## ÇALIŞTIIIIR
val_ = df.loc[(df["date"] >= "2015-10-01"), :]

cols = [col for col in df.columns if col not in ['date', 'ID', "item_cnt_day","year", "item_id", "shop_id", "item_category_id" ]]

Y_train = train_['item_cnt_day']
X_train = train_[cols]

Y_val = val_['item_cnt_day']
X_val = val_[cols]

In [31]:
import itertools
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima_model import ARIMA
# from statsmodels.tsa.holtwinters import ExponentialSmoothing
# from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.tsa.api as smt
from statsmodels.tsa.statespace.sarimax import SARIMAX
warnings.filterwarnings('ignore')


In [ ]:

def is_stationary(y):

    # "HO: Non-stationary"
    # "H1: Stationary"

    p_value = sm.tsa.stattools.adfuller(Y_train)[1]
    if p_value < 0.05:
        print(F"Result: Stationary (H0: non-stationary, p-value: {round(p_value, 3)})")
    else:
        print(F"Result: Non-Stationary (H0: non-stationary, p-value: {round(p_value, 3)})")

is_stationary(Y_train)

In [26]:
lgb_params = {'metric': {'mae'},
              'num_leaves': 10,
              'learning_rate': 0.02,
              'feature_fraction': 0.8,
              'max_depth': 5,
              'verbose': 0,
              'num_boost_round': 1000,
              'early_stopping_rounds': 200,
              'nthread': -1}


lgbtrain = lgb.Dataset(data=X_train, label=Y_train, feature_name=cols)
lgbval = lgb.Dataset(data=X_val, label=Y_val, reference=lgbtrain, feature_name=cols)

model = lgb.train(lgb_params, lgbtrain,
                  valid_sets=[lgbtrain, lgbval],
                  num_boost_round=lgb_params['num_boost_round'],
                  early_stopping_rounds=lgb_params['early_stopping_rounds'],
                  feval=lgbm_smape,
                  verbose_eval=100)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 5.321672 seconds.
You can set `force_col_wise=true` to remove the overhead.
Training until validation scores don't improve for 200 rounds
[100]	training's l1: 0.441161	training's SMAPE: nan	valid_1's l1: 0.44982	valid_1's SMAPE: nan
[200]	training's l1: 0.439179	training's SMAPE: nan	valid_1's l1: 0.449273	valid_1's SMAPE: nan
Early stopping, best iteration is:
[1]	training's l1: 0.445805	training's SMAPE: nan	valid_1's l1: 0.450449	valid_1's SMAPE: nan


In [27]:
T = test.loc[test.item_cnt_day.isna()]
X_test = test[cols]

AttributeError: 'DataFrame' object has no attribute 'item_cnt_day'

In [ ]:
y_pred_lgb = model.predict(X_test).clip(0., 20.)

In [ ]:
preds = pd.DataFrame(y_pred_lgb, columns=['item_cnt_month'])
preds.to_csv('submission.csv',index_label='ID')